# T1 SEMANTIC SEGMENTATION TRAINING

Small example on how to train Pointcept for semantic segmentation

In [ ]:
import os

root_folder = os.path.abspath('..')
data_folder_name = os.path.join(root_folder, 'test_data')

print(root_folder)

## LIBRARIES

In [ ]:
#IMPORT PACKAGES
from pathlib import Path
import sys
sys.path.insert(0, root_folder)
sys.path.insert(0, os.path.join(root_folder, "scripts"))
sys.path.insert(0, os.path.join(root_folder, 'thirdparty', 'pointcept'))
print(sys.path)
import numpy as np
import laspy
from geomapi.utils import geometryutils as gmu
import torch

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

## LOAD AND REMAP CLASSES

In [ ]:
import json

classes_file = os.path.join(data_folder_name, '_classes.json')

# Read the JSON file
with open(classes_file, 'r') as file:
    json_data = json.load(file)

classes_list = json_data['classes']

print(classes_list)

remapped_classes_ids = {}

for class_entry in classes_list:
    remapped_classes_ids[int(class_entry["id"])] = int(class_entry["temp_id"])

print(remapped_classes_ids)

## INPUT DATA CONVERSION

Preprocessing of input data

In [ ]:
def handle_process(file_name, output_folder):
    
    print(file_name)

    scene_id = os.path.basename(file_name)

    name, ext = os.path.splitext(scene_id)
    
    if ext not in  [".las", ".laz"]:
        return

    # Read LAS/LAZ
    # populate dict
    las = laspy.read(file_name)
    print(list(las.point_format.dimension_names))

    pcd = gmu.las_to_pcd(las)
    pcd.estimate_normals()
    pcd.orient_normals_to_align_with_direction()
    
    coords = np.stack([las.x, las.y, las.z], axis=1)
    colors = np.stack([las.red / 256, las.green / 256, las.blue / 256], axis=1).astype(np.uint8)
    normals = np.asarray(pcd.normals)
    
    classes = []

    for class_id in las.classes.astype(int):
        classes.append(remapped_classes_ids[class_id])

    save_dict = dict(coord=coords, color=colors, normal=normals, scene_id=scene_id, semantic_gt=np.array(classes).astype(int))

    torch.save(save_dict, os.path.join(output_folder, f"{name}.pth"))


training_las_folder = os.path.join(data_folder_name, 't1_data', 'input', 'train')
training_output_folder = os.path.join(data_folder_name, 't1_data', 'train')

validation_las_folder = os.path.join(data_folder_name, 't1_data', 'input', 'val')
validation_output_folder = os.path.join(data_folder_name, 't1_data', 'val')

os.makedirs(training_output_folder, exist_ok=True)
os.makedirs(validation_output_folder, exist_ok=True)

for file_name in os.listdir(training_las_folder):
    handle_process(os.path.join(training_las_folder, file_name), training_output_folder)

for file_name in os.listdir(validation_las_folder):
    handle_process(os.path.join(validation_las_folder, file_name), validation_output_folder)

## TRAINING

Training using Point Transformer V3

In [ ]:
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
    default_setup,
)
from pointcept.engines.train import TRAINERS
from pointcept.engines.launch import launch


def main_worker(cfg):
    cfg = default_setup(cfg)
    trainer = TRAINERS.build(dict(type=cfg.train.type, cfg=cfg))
    trainer.train()

config_path = os.path.join(data_folder_name, 't1_data', 'config.py')
save_path = os.path.join(data_folder_name, 't1_data')
weights = os.path.join(data_folder_name, 't1_data', 'model', 'model_best.pth')

cfg = default_config_parser(str(config_path), {'save_path': str(save_path), 'weight': str(weights)})

launch(
    main_worker,
    num_gpus_per_machine=1,
    num_machines=1,
    machine_rank=0,
    dist_url='auto',
    cfg=(cfg,),
)